<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Example 4.1

import numpy as np
import pandas as pd


file_name = '/content/PV Modeling Book Data Source.xls'
sheet_name = 'Source 7'


G = pd.read_excel(file_name, sheet_name=sheet_name, usecols="A", nrows=8761)
T = pd.read_excel(file_name, sheet_name=sheet_name, usecols="B", nrows=8761)
L = pd.read_excel(file_name, sheet_name=sheet_name, usecols="D", nrows=8761)


G = pd.to_numeric(G.squeeze(), errors='coerce')
T = pd.to_numeric(T.squeeze(), errors='coerce')
L = pd.to_numeric(L.squeeze(), errors='coerce')



PV_Wp = 2500
Battery_SOCmax = 1400
PV_eff = 0.16
V_B = 12
Inv_RP = 2500
DOD = 0.8
Charge_eff = 0.8
Alpha = 0.05
Wire_eff = 0.98

SOCmax = Battery_SOCmax
SOCmin = SOCmax * (1 - DOD)


P_Ratio = (PV_Wp * (G / 1000)) / Inv_RP
Inv_eff = 97.644 - (P_Ratio * 1.995) - (0.445 / P_Ratio.replace(0, np.nan))
Inv_eff = Inv_eff.fillna(0)


E_PV = ((PV_Wp * (G / 1000)) - (Alpha * (T - 25))) * Wire_eff * Inv_eff


E_net = E_PV - L


SOCi = SOCmax
SOCf = []
Deff = []
Dampf = []


for ED in E_net:
    SOC = ED + SOCi

    if SOC > SOCmax:
        Dampi = SOC - SOCmax
        Defi = 0
        SOCi = SOCmax

    elif SOC < SOCmin:
        SOCi = SOCmin
        Defi = SOC - SOCmin
        Dampi = 0

    else:
        Defi = 0
        Dampi = 0
        SOCi = SOC

    SOCf.append(SOCi)
    Deff.append(Defi)
    Dampf.append(Dampi)


SOCf = np.array(SOCf)
Deff = np.array(Deff)
Dampf = np.array(Dampf)
SOC_per = SOCf / SOCmax
LLP_calculated = abs(np.sum(Deff)) / np.sum(L)


print("Loss of Load Probability (LLP):", LLP_calculated)

Loss of Load Probability (LLP): 0.0
